In [48]:
from google.colab import drive
from os.path import join 

ROOT = "/content/gdrive/"
drive.mount(ROOT)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## 인쇄체 데이터에 대해 rawdata 및 전처리 수행한 최종 데이터 얻음

#### 인쇄체 데이터 파악

In [49]:
import json
def getListOfPrintedFileNames(listOfPrintedFileNames):
    printedMetaFile_location = '/content/gdrive/Shareddrives/machine_learning_in_practice/한국어 글자체 이미지/02.인쇄체/printed_data_info.json'

    with open(printedMetaFile_location, 'r') as f:
        p_json_data = json.load(f)

    p_ClassToFileNames, p_WordsToFileNames= {}, {}
    p_FIleNameToClass = {} # 파일명 : json_data['images'][idx]['file_name']
    p_syllableDataLength = 0

    for idx, img_data in enumerate(p_json_data['images']):
        # idx 번째 이미지 데이터들에 대해 {클래스 : [파일명, 파일명, ...]}
        class_ = p_json_data['annotations'][idx]['text']
        fileName = p_json_data['images'][idx]['file_name']
        if len(class_) == 1:    
            # 한 글자인 경우만 ClassToFileNames에 넣기
            if class_ not in p_ClassToFileNames.keys(): # idx번째 이미지의 클래스가 새로운 클래스이면
                p_ClassToFileNames[class_] = [] # 빈 리스트 생성
            p_ClassToFileNames[class_].append(fileName)
            p_syllableDataLength += 1
            listOfPrintedFileNames.append(fileName)

        else:
            # 단어, 문장의 경우 WordsToFileNames에 넣기
            if class_ not in p_WordsToFileNames.keys(): # idx번째 이미지의 클래스가 새로운 클래스이면
                p_WordsToFileNames[class_] = [] # 빈 리스트 생성
            p_WordsToFileNames[class_].append(fileName)
    for idx, img_data in enumerate(p_json_data['images']):
        fileName = p_json_data['images'][idx]['file_name']
        class_ = p_json_data['annotations'][idx]['text']
        p_FIleNameToClass[fileName] = class_

    print("*** Printed Data Description ***")
    print("\t전체 데이터 수 : ", len(p_json_data['images']) )# 여기에는 문장 데이터, 단어 데이터, 음절 데이터 모두 포함됨
    print("\t음절 클래스 수 : ", len(p_ClassToFileNames) ) # 한국어 글자 종류는 11,172가지.
    print("\t단어/문장 클래스 수: ", len(p_WordsToFileNames))
    print("\t음절 데이터 수 : ", p_syllableDataLength)
    print("\t단어/문장 데이터 수 : ", len(p_json_data['images']) - p_syllableDataLength)
    print(f"\t한 음절 당 평균 {p_syllableDataLength/len(p_ClassToFileNames)} 개 손글씨 데이터 존재함")

    return p_ClassToFileNames, p_FIleNameToClass

In [50]:
# 인쇄체 음절 이미지의 파일명  for pretrain
listOfPrintedFileNames = []

p_ClassToFileNames, p_FIleNameToClass = getListOfPrintedFileNames(listOfPrintedFileNames)

print("인쇄체 data 수 : ", len(listOfPrintedFileNames))

*** Printed Data Description ***
	전체 데이터 수 :  878100
	음절 클래스 수 :  11172
	단어/문장 클래스 수:  6054
	음절 데이터 수 :  575350
	단어/문장 데이터 수 :  302750
	한 음절 당 평균 51.499283924095955 개 손글씨 데이터 존재함
인쇄체 data 수 :  575350


#### 자주 사용하는 한국어 + 잘 안쓰는 한국어 총 1268 종류 글자 및 파일 파악 

In [ ]:
import pandas as pd

classFile = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/한국어 학습용 어휘 목록.xls'
df = pd.read_excel(classFile)

df.head()

,순위,단어,품사,풀이,등급
0,1195.0,가게,명,NaN,A
1,898.0,가격03,명,價格,B
2,2986.0,가구03,명,家口,C
3,7434.0,가구04,명,家具,B
4,4964.0,가까워지다,동,NaN,B


In [ ]:
word_list = df["단어"].tolist()
# word_list 수정,  '가구04', '도쿄(동경)' 가공
for idx, word in enumerate(word_list):
    if len(word) > 2:
        if word[-2] in ('0', '1', '2', '3'):
            word_list[idx] = word[:-2]
    if word[-1] == ')':
        del word_list[idx]
        
len(word_list)

5963

In [ ]:
syllable_set = set()
for word in word_list:
    for syllable in word:
        syllable_set.add(syllable)
len(syllable_set)

dump_set = {'0', '1', '8'}

easy_syllable_set = syllable_set - dump_set
len(easy_syllable_set)

971

In [ ]:
whole_syllable_set = set(p_ClassToFileNames.keys())
len(whole_syllable_set)

11172

In [ ]:
difficult_syllable_set = whole_syllable_set - easy_syllable_set
len(difficult_syllable_set)

10201

In [ ]:
import random

difficult_syllable_list = list(difficult_syllable_set)
classList = []

randIdx = random.randint(0, len(difficult_syllable_set))

difficult_syllable_choice = set()

for i in range(300):
    randIdx = random.randint(0, len(difficult_syllable_set))
    difficult_syllable_choice.add(randIdx)
len(difficult_syllable_choice)

idxs = list(difficult_syllable_choice)


difficult_syllable_classes = set()
for idx in idxs:
    difficult_syllable_classes.add(difficult_syllable_list[idx])
len(difficult_syllable_classes)

297

In [51]:
syllableClass = list(difficult_syllable_classes | easy_syllable_set)

import csv  
with open('/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain/classes.csv', 'w', newline='') as f: 
    writer = csv.writer(f) 
    writer.writerow(syllableClass) 

print("글씨 수 : ", len(syllableClass))

글씨 수 :  1268


In [ ]:
newClassToFileNames = {}
for syllable in difficult_syllable_classes:
    newClassToFileNames[syllable] = p_ClassToFileNames[syllable]

for syllable in easy_syllable_set:
    newClassToFileNames[syllable] = p_ClassToFileNames[syllable]

print("잘 쓰는 글자 : ", easy_syllable_set) # 국립국어연구원에서 2003년 5월에 발표한 한국어 학습용 어휘 목록에서 따옴
print("잘 안쓰는 글자 : ", difficult_syllable_classes) # 위의 꺼의 여집합 중 일부 ( 비율 거의 동일하게 함)
print("잘 쓰는 글자와 잘 안쓰는 글자 총 클래스 : ", len(newClassToFileNames))

잘 쓰는 글자 :  {'멋', '쌓', '팎', '렇', '름', '잇', '냉', '몽', '선', '끓', '좋', '민', '좌', '뿐', '뿌', '컬', '에', '닐', '말', '복', '션', '접', '왜', '볍', '력', '무', '끼', '뜯', '유', '푸', '팀', '패', '젖', '훌', '쿠', '닷', '싹', '쉽', '넉', '핏', '찌', '늬', '깎', '뻐', '져', '램', '준', '터', '확', '닭', '매', '은', '랍', '런', '량', '뤄', '끝', '담', '굵', '람', '쨌', '귀', '횟', '축', '할', '깜', '월', '작', '볶', '니', '별', '균', '쓰', '찢', '칭', '뼉', '헤', '햄', '꽉', '삶', '억', '우', '층', '굉', '앞', '쉰', '림', '용', '쟤', '벌', '둘', '뺨', '릎', '퓨', '낮', '론', '땅', '얹', '씹', '덮', '값', '낳', '태', '효', '협', '료', '몹', '흉', '덜', '랑', '딩', '쩌', '닿', '탕', '색', '낭', '왼', '함', '또', '즘', '실', '듣', '채', '앨', '플', '천', '엘', '브', '델', '엄', '칠', '싣', '곤', '탁', '콩', '찾', '굽', '손', '숭', '덧', '맡', '몰', '뼈', '잠', '게', '깊', '떻', '툼', '익', '장', '떼', '껌', '씬', '욱', '박', '빼', '자', '걱', '루', '팝', '며', '맵', '낡', '던', '올', '법', '콘', '증', '철', '놔', '밥', '떨', '찍', '졸', '립', '큼', '밉', '빌', '몬', '꺾', '뭘', '낌', '흰', '여', '듭', '외', '김', '느', '잣', '이', '헌', '덩', '누', '섯', '쟁', '않', '롯', '셔'

#### FileNameToClass.pickle - key를 한정 (기존 FileNameToClass는 11172개의 class 모두 담고 있음 => 1268개 클래스로 축소)

In [ ]:
classes = newClassToFileNames.keys()
new_pFileNameToClass = dict()
for fileName in p_FIleNameToClass.keys():
    if p_FIleNameToClass[fileName] not in classes:
        continue
    new_pFileNameToClass[fileName] = p_FIleNameToClass[fileName]

print("클래스 축소 전 데이터 수 : ", len(p_FIleNameToClass))
print("클래스 축소 후 데이터 수 : ", len(new_pFileNameToClass))

클래스 축소 전 데이터 수 :  878100
클래스 축소 후 데이터 수 :  80150


#### raw_printed.zip | 1268개 클래스에 해당하는 이미지 저장

In [ ]:
!unzip '/content/gdrive/Shareddrives/machine_learning_in_practice/한국어 글자체 이미지/02.인쇄체/01_printed_syllable_images.zip' -d .

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./syllable/00508191.png  
  inflating: ./syllable/00508198.png  
  inflating: ./syllable/00508199.png  
  inflating: ./syllable/00508204.png  
  inflating: ./syllable/00508209.png  
  inflating: ./syllable/00508210.png  
  inflating: ./syllable/00508214.png  
 extracting: ./syllable/00508215.png  
  inflating: ./syllable/00508219.png  
  inflating: ./syllable/00508220.png  
  inflating: ./syllable/00508229.png  
  inflating: ./syllable/00508234.png  
  inflating: ./syllable/00508235.png  
  inflating: ./syllable/00508237.png  
  inflating: ./syllable/00508239.png  
  inflating: ./syllable/00508244.png  
  inflating: ./syllable/00508246.png  
 extracting: ./syllable/00508249.png  
  inflating: ./syllable/00508251.png  
  inflating: ./syllable/00508253.png  
  inflating: ./syllable/00508255.png  
  inflating: ./syllable/00508257.png  
  inflating: ./syllable/00508261.png  
  inflating: ./syllable/00508271.png  
  inflating: ./syllable/0050

In [ ]:
import os

path = '/content/syllable'
file_list = os.listdir(path)

# new_pFileNameToClass 는 1268개 클래스에 해당하는 image filename 들만 key 로 있음
newFileList = []
newClassList = []
for filename in file_list:
    try: # new_pFileNameToClass 이용함 key 에러 뜨면 넘겨야 함
        class_ = new_pFileNameToClass[filename]
        newFileList.append(filename)
        newClassList.append(class_)
    except KeyError:
        # print(filename)
        pass
print(len(newFileList))
print(len(newClassList))

60465
60465


In [ ]:
import csv

filepath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain/fileName.csv'
with open(filepath, 'w', newline='') as f: 
    writer = csv.writer(f) 
    writer.writerow(newFileList)

In [ ]:
import os
import zipfile

new_zips= zipfile.ZipFile('raw_printed.zip', 'w') # 
num_zipfile = 0

for folder, subfolders, files in os.walk('/content/syllable'): 
    for file in files:
        try:
            new_pFileNameToClass[file] # 1268 클래스에 속하는 데이터만 zip함  총 112300개
            new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './'), compress_type = zipfile.ZIP_DEFLATED)
            num_zipfile += 1
        except KeyError:
            pass
new_zips.close()
print(num_zipfile)

60465


In [ ]:
import shutil

savePath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain'

shutil.move('/content/raw_printed.zip', savePath)

'/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/raw_printed.zip'

#### croppedImg 생성

In [ ]:
!unzip '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/raw_printed.zip' -d ./rawdata

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./rawdata/syllable/00141873.png  
  inflating: ./rawdata/syllable/00228232.png  
  inflating: ./rawdata/syllable/00050363.png  
  inflating: ./rawdata/syllable/00296008.png  
  inflating: ./rawdata/syllable/00308880.png  
  inflating: ./rawdata/syllable/00473574.png  
  inflating: ./rawdata/syllable/00092955.png  
  inflating: ./rawdata/syllable/00192494.png  
  inflating: ./rawdata/syllable/00115889.png  
  inflating: ./rawdata/syllable/00325591.png  
  inflating: ./rawdata/syllable/00083227.png  
  inflating: ./rawdata/syllable/00272463.png  
  inflating: ./rawdata/syllable/00038818.png  
  inflating: ./rawdata/syllable/00249872.png  
  inflating: ./rawdata/syllable/00330932.png  
  inflating: ./rawdata/syllable/00255102.png  
  inflating: ./rawdata/syllable/00140215.png  
  inflating: ./rawdata/syllable/00417378.png  
  inflating: ./rawdata/syllable/00430143.png  
  inflating: ./rawdata/syllable/00155671.png  
  inflating: ./rawdata/s

In [ ]:
import cv2
import csv
import os
import numpy as np
from PIL import Image, ImageOps
import matplotlib.image as img 

savePath = '/content/preprocessed' 

path = '/content/rawdata/syllable'
file_list_ = os.listdir(path)

for filename in file_list_:
    filePath = path + '/' + filename
    image=Image.open(filePath)
    image.load()
    imageSize = image.size
    
    # remove alpha channel
    invert_im = image.convert("RGB") 
    
    # invert image (so that white is 0)
    invert_im = ImageOps.invert(invert_im)
    imageBox = invert_im.getbbox()
    
    cropped=image.crop(imageBox)
    print(filePath, "Size:", imageSize, "New Size:", imageBox)
    cropped.save(savePath + '/' + filename)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/rawdata/syllable/00141873.png Size: (94, 114) New Size: (8, 18, 82, 109)
/content/rawdata/syllable/00228232.png Size: (101, 123) New Size: (7, 25, 96, 118)
/content/rawdata/syllable/00050363.png Size: (105, 127) New Size: (18, 26, 86, 122)
/content/rawdata/syllable/00296008.png Size: (100, 105) New Size: (8, 10, 82, 100)
/content/rawdata/syllable/00308880.png Size: (101, 105) New Size: (7, 11, 94, 100)
/content/rawdata/syllable/00473574.png Size: (102, 134) New Size: (13, 38, 93, 129)
/content/rawdata/syllable/00092955.png Size: (104, 106) New Size: (8, 6, 97, 101)
/content/rawdata/syllable/00192494.png Size: (100, 134) New Size: (13, 38, 94, 129)
/content/rawdata/syllable/00115889.png Size: (98, 109) New Size: (8, 9, 89, 104)
/content/rawdata/syllable/00325591.png Size: (101, 108) New Size: (7, 9, 95, 103)
/content/rawdata/syllable/00083227.png Size: (91, 128) New Size: (11, 33, 80, 123)
/content/rawdata/syllable/00272463.png Size: (105, 11

In [ ]:
import os
import zipfile

new_zips= zipfile.ZipFile('preprocessed.zip', 'w')
 
for folder, subfolders, files in os.walk('/content/preprocessed/'): 
    for file in files:
        new_zips.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), './'), compress_type = zipfile.ZIP_DEFLATED)

new_zips.close()

In [ ]:
import shutil

savePath = '/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain'

shutil.move('preprocessed.zip', savePath)

'/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/GAN/data/pretrain/preprocessed.zip'

## github 커밋

In [ ]:
MY_GOOGLE_DRIVE_PATH = "/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital"
%cd "{MY_GOOGLE_DRIVE_PATH}"

/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital


In [ ]:
!git config --global user.email dkwjd0824@khu.ac.kr  # 이메일 입력 ex) qhrqufdlek@naver.com
!git config --global user.name  hyeneung #깃헙 아이디 입력 ex)luckydipper
!git pull

Already up to date.


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   GAN/Parsing.ipynb
	modified:   GAN/model/SelfAttention_Wasserstein/discriminator.pth
	modified:   GAN/model/SelfAttention_Wasserstein/generator.pth
	modified:   GAN/selfAttentionGAN.ipynb
	deleted:    "object_detection/20211119 \341\204\211\341\205\265\341\206\257\341\204\214\341\205\245\341\206\253\341\204\200\341\205\265\341\204\200\341\205\250\341\204\222\341\205\241\341\206\250\341\204\211\341\205\263\341\206\270 \341\204\211\341\205\265\341\206\257\341\204\211\341\205\263\341\206\270 \341\204\217\341\205\251\341\204\203\341\205\263 (1).ipynb"
	deleted:    "object_detection/20211119 \354\213\244\354\240\204\352\270\260\352\263\204\355\225\231\354\212\265 \354\213\244\354\212\265 \354\275\224\353\223\234 (1).ipynb"
	deleted:    object_detec

In [ ]:
!git add GAN/PrintedDataset.ipynb

In [ ]:
!git commit -m"[ADD] Reload printedDataset"

[main dbfc005] [ADD] Reload printedDataset
 1 file changed, 1 insertion(+)
 create mode 100644 GAN/PrintedDataset.ipynb


In [ ]:
!git push

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 110.88 KiB | 3.82 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/PILGI-Analog-To-Digital/Analog-PILGI-to-DIgital.git
   d9a82c8..dbfc005  main -> main
